In [ ]:
import pandas as pd
df_inventory = pd.read_csv('inventory.csv')
df_inbound = pd.read_csv('inbound.csv')
df_outbound = pd.read_csv('outbound.csv')
df_cap = pd.read_csv('capacity.csv')

In [ ]:
# date format according to inventory dataset month/day/year
def total_cap(date = '1/1/2025'):
    [month,day,year] = date.split('/')
    df_latest = df_inventory[df_inventory['BALANCE_AS_OF_DATE'] == date]

    if month == '1':
            latest_month = 12
            year = int(year) - 1
    else: 
        latest_month = int(month) - 1
    

    # date format for inventory
    format_date_inventory = str(latest_month) + '/' + str(year)

    # date format for inbound and outbound
    if int(month) < 10:
        full_date_format = str(year) + '/' + '0' + str(month) + '/' + day
        format_date_inout = str(year) + '/' + '0' + str(month)
    else:
        full_date_format = str(year) + '/' + str(month) + '/' + day
        format_date_inout = str(year) + '/' + str(month)
    
    # check end of month
    if df_latest.shape[0] == 0:
        
        # dataframe snapshot
        df_latest = df_inventory[df_inventory['MONTH_YEAR'] == format_date_inventory]
        
        # cap of snapshot
        count_cap_all = df_latest.groupby('PLANT_NAME')['UNRESRICTED_STOCK'].sum()
        china_cap_MT = count_cap_all['CHINA-WAREHOUSE'] * 0.001
        singapore_cap_MT = count_cap_all['SINGAPORE-WAREHOUSE'] * 0.001

        # dataframe inbound and outbound in that month
        current_month_inbound = df_inbound[df_inbound['MONTH_YEAR'] == format_date_inout]
        current_month_outbound = df_outbound[df_outbound['MONTH_YEAR'] == format_date_inout]

        # dataframe range interval from first day to the given day
        inbound_range = current_month_inbound[current_month_inbound['INBOUND_DATE'] <= full_date_format]
        outbound_range = current_month_outbound[current_month_outbound['OUTBOUND_DATE'] <= full_date_format]
        
        # inbound and outbound
        sum_inbound_all = inbound_range.groupby('PLANT_NAME')['NET_QUANTITY_MT'].sum()
        sum_outbound_all = outbound_range.groupby('PLANT_NAME')['NET_QUANTITY_MT'].sum()
        
        # inbound seperate plants
        sum_inbound_china = sum_inbound_all.get('CHINA-WAREHOUSE', 0)
        sum_inbound_singapore = sum_inbound_all.get('SINGAPORE-WAREHOUSE', 0)
        
        # outbound seperate plants
        sum_outbound_china = sum_outbound_all.get('CHINA-WAREHOUSE', 0)
        sum_outbound_singapore = sum_outbound_all.get('SINGAPORE-WAREHOUSE', 0)

        # diff inbound and outbound
        diff_china = sum_inbound_china - sum_outbound_china
        diff_singapore = sum_inbound_singapore - sum_outbound_singapore

        # total cap of each plant
        china_cap = round(int(china_cap_MT + diff_china),2)
        singapore_cap = round(int(singapore_cap_MT + diff_singapore),2)

        china_cap_KT = round(china_cap_MT *  0.0011023113109244, 4)
        singapore_cap_KT = round(singapore_cap_MT *  0.0011023113109244, 4)

        #print(f"chinal capacity at {date}: {china_cap_KT} KT" )
        #print(f"singapore capacity at {date}: {singapore_cap_KT} KT")
    else:
         # dataframe snapshot
        df_latest = df_inventory[df_inventory['MONTH_YEAR'] == format_date_inventory]
        
        # cap of snapshot
        count_cap_all = df_latest.groupby('PLANT_NAME')['UNRESRICTED_STOCK'].sum()
        china_cap_MT = count_cap_all['CHINA-WAREHOUSE'] * 0.001
        singapore_cap_MT = count_cap_all['SINGAPORE-WAREHOUSE'] * 0.001

        china_cap_KT = round(china_cap_MT *  0.0011023113109244, 4)
        singapore_cap_KT = round(singapore_cap_MT *  0.0011023113109244, 4)

        #print(f"chinal capacity at {date}: {china_cap_KT} KT" )
        #print(f"singapore capacity at {date}: {singapore_cap_KT} KT")

    
    data = [
            {'Date': date, 'Warehouse': 'CHINA-WAREHOUSE (KT)', 'Capacity_KT': china_cap_KT},
            {'Date': date, 'Warehouse': 'SINGAPORE-WAREHOUSE (KT)', 'Capacity_KT': singapore_cap_KT}
        ]
    return data


    

In [ ]:
# Initialize an empty list to store the transformed rows
transformed_data = []

# Iterate through each row of the original DataFrame
for index, row in df_cap.iterrows():
    date = row['Date']

    # Process CHINA-WAREHOUSE data
    transformed_data.append({
        'Label': 'Used',
        'Date': date,
        'Cap': row['CHINA-WAREHOUSE'],
        'Warehouse': 'china'
    })
    transformed_data.append({
        'Label': 'Remain',
        'Date': date,
        'Cap': row['DIFFERENCE_CHINA'],
        'Warehouse': 'china'
    })

    # Process SINGAPORE-WAREHOUSE data
    transformed_data.append({
        'Label': 'Used',
        'Date': date,
        'Cap': row['SINGAPORE-WAREHOUSE'],
        'Warehouse': 'singapore'
    })
    transformed_data.append({
        'Label': 'Remain',
        'Date': date,
        'Cap': row['DIFFERENCE_SINGAPORE'],
        'Warehouse': 'singapore'
    })

# Create the new DataFrame from the transformed data list
new_df = pd.DataFrame(transformed_data)
new_df.to_csv('capacity_transformed.csv', index=False)